In [1]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# -*- coding: utf-8 -*-

import json

f = open("data.json", "r")
txt = f.read()

def removeNonAscii(s): return "".join(i for i in s if ord(i)<128)

n = removeNonAscii(txt)

data = json.loads(n)

print(data)


In [ ]:
y = pd.DataFrame.from_dict(data['rss']['channel']['item']).drop(columns = ["comment_status", "description", "guid", "is_sticky", "menu_order", "ping_status", "post_parent", "post_password", "post_id", "post_type", "postmeta", "status", "post_name", "post_date", "post_date_gmt"])

In [ ]:
new = []
for thing in y['creator']:
    new.append(thing["__cdata"])
    
print(new)

y['creator'] = new

    

In [ ]:
new = []
for thing in y['encoded']:
    new.append(thing[0]["__cdata"])
    
print(new)

y['encoded'] = new


In [ ]:
y['category']
authors = []
categories = []

for line in y['category']:
    current_authors = []
    current_categories = []
    for dic in line:
        if dic['_domain'] == 'author':
            current_authors.append(dic['_nicename'])
        if dic['_domain'] == 'category':
            current_categories.append(dic['_nicename'])
            
    authors.append(current_authors)
    categories.append(current_categories)
        

In [ ]:
y['authors'] = authors
y['categories'] = categories

y.head()

In [ ]:
y.rename(columns={"encoded": "encoded_text"}, inplace=True)
y.drop(columns=["category"], inplace=True)

In [ ]:
y.head()

In [ ]:
y.to_csv("xml_data.csv")

In [ ]:
count = 0
for line in y['encoded_text']:
    if "img" in line:
        count +=1
        
print(count)

count = 0
for line in y['encoded_text']:
    if "src" in line:
        count +=1
        
print(count)

In [ ]:
final = pd.read_csv("merged.csv")
final.head()
def convertPercent(percent):
    return float(str(percent).strip('%'))/100

def percentToFloat(name):
    arr = final[name].to_numpy()
    new = []
    for value in arr:
        new.append(convertPercent(value))
    final[name] = new

percentToFloat("bounce_rate")
percentToFloat("exit_percent")

final.head()


In [ ]:
print(final.loc[[7]].to_numpy())

In [ ]:
link = final.columns.get_loc("link")
views = final.columns.get_loc("views")
unique_views = final.columns.get_loc("unique_views")
entrances = final.columns.get_loc("entrances")
avg_time = final.columns.get_loc("avg_time")
bounce_rate = final.columns.get_loc("bounce_rate")
exit_percent = final.columns.get_loc("exit_percent")

def combine(one, two):
    one.reset_index(drop=True, inplace=True)
    two.reset_index(drop=True, inplace=True)
    
    one = one.to_numpy()
    two = two.to_numpy()
    
#     print(one)
    
#     print(two)


#     if one[0][link] != two[0][link]:
#         return
#     else:
    one_fraction = one[0][views] / (one[0][views] + two[0][views])
    two_fraction = two[0][views] / (one[0][views] + two[0][views])

    one[0][views] = one[0][views] + two[0][views]
    one[0][unique_views] = one[0][unique_views] + two[0][unique_views]
    one[0][entrances] = one[0][entrances] + two[0][entrances]
    one[0][avg_time] = one[0][avg_time] * one_fraction + two[0][avg_time] * two_fraction
    one[0][bounce_rate] = one[0][bounce_rate] * one_fraction + two[0][bounce_rate] * two_fraction
    one[0][exit_percent] = one[0][exit_percent] * one_fraction + two[0][exit_percent] * two_fraction

    x = pd.DataFrame(one, columns=final.columns)

    return x
    

# head = combine(final.loc[[7]], final.loc[[8]])

# head
# final.loc[[7]].to_numpy()[0][link]
    

In [ ]:
def combinegroup(df):
    df.reset_index(drop=True, inplace=True)
    if len(df) == 1:
        return df
    
    
    if len(df) == 2:
        return combine(df.loc[[0]], df.loc[[1]])
    
    new = combine(df.loc[[0]], df.loc[[1]])
    for i in range(2, len(df)):
        new = combine(new, df.loc[[i]])
    
    return new

test = final.loc[final['link'] == 'https://dailybruin.com/2020/06/15/ucla-announces-fall-quarter-plans-for-classes-housing-health-precautions/']
test

har = combinegroup(test)


In [ ]:
final.head(178)

In [ ]:

def castToFloat(name):
    arr = final[name].to_numpy()
    new = []
    for value in arr:
        new.append(float(str(value).replace(",", "")))
    final[name] = new

castToFloat("views")
castToFloat("unique_views")
castToFloat("entrances")

def convert_time(time):
#     print(time)
    if str(time) == 'nan' or str(time) == 'None':
        return "";
    mins = int(str(time)[2:4])
    second = int(str(time)[5:])
    total_seconds = mins * 60 + second
    return total_seconds
    
arr = final['avg_time'].to_numpy()
new = []
for value in arr:
    new.append(convert_time(value))
final['avg_time'] = new

final.head()
    








In [2]:
final = pd.read_csv("merged.csv")

def convertPercent(percent):
    return float(str(percent).strip('%'))/100

def percentToFloat(name):
    arr = final[name].to_numpy()
    new = []
    for value in arr:
        new.append(convertPercent(value))
    final[name] = new

percentToFloat("bounce_rate")
percentToFloat("exit_percent")

final.head()


,Unnamed: 0,page,views,unique_views,avg_time,entrances,bounce_rate,exit_percent,page_value,title,link,pubDate,creator,encoded_text,authors,categories,image_url
0,0,/,90941.0,74000.0,0:01:24,63825.0,0.004599,0.004502,$0.00,Graduation Issue 2020,https://gradissue2020.dailybruin.com/,"Fri, 12 Jun 2020 17:09:01 +0000",Camayak_180,We are past the tipping point. Where do we go ...,['camayak_180'],['interactive-page'],gradissue2020.dailybruin.com/smol.gradissue-pr...
1,1,/2020/06/04/students-sign-petition-to-fire-ucl...,41497.0,38105.0,0:04:55,37002.0,0.009070,0.008916,$0.00,Students sign petition to fire UCLA lecturer a...,https://dailybruin.com/2020/06/04/students-sig...,"Thu, 04 Jun 2020 23:47:55 +0000",Camayak_8ce8660ac1,This post was updated June 5 at 1:35 p.m. ...,['cap-kari-lau'],"['campus', 'news']",https://wp.dailybruin.com/images/2020/06/web.n...
2,2,/2020/07/11/album-review-juice-wrld-proves-leg...,24298.0,18392.0,0:02:35,18345.0,0.007960,0.007512,$0.00,Album review: Juice WRLD proves 'Legends Never...,https://dailybruin.com/2020/07/11/album-review...,"Sat, 11 Jul 2020 23:00:54 +0000",Camayak_2fac78eb7b,Emo-rap artist Juice WRLDs first posthumous al...,['cap-kanishka-mehra'],"['arts-entertainment', 'ae-columns', 'music']",https://wp.dailybruin.com/images/2020/07/web.a...
3,3,/2020/10/24/student-dies-after-falling-from-we...,20347.0,17904.0,0:04:06,15854.0,0.008516,0.008216,$0.00,Student dies after falling from Westwood apart...,https://dailybruin.com/2020/10/24/student-dies...,"Sat, 24 Oct 2020 21:23:40 +0000",Camayak_351705866e,A student died early Saturday morning after f...,"['cap-elizabeth-sherwood', 'cap-bernard-mendez']","['news', 'westwood']",https://wp.dailybruin.com/images/2020/10/news....
4,4,/2020/07/16/ucs-2020-acceptance-rate-rose-to-6...,10921.0,10367.0,0:03:30,10004.0,0.009128,0.008958,$0.00,"UC's 2020 acceptance rate rose to 69.5%, admit...",https://dailybruin.com/2020/07/16/ucs-2020-acc...,"Fri, 17 Jul 2020 00:56:09 +0000",Camayak_bef6dcd253,This post was updated July 16 at 8:18 p.m. ...,['cap-genesis-qu'],"['campus', 'news', 'university-of-california']",https://wp.dailybruin.com/images/2020/07/uclas...


In [3]:
print(final.loc[[7]].to_numpy())

[[7
  '/2020/06/15/ucla-announces-fall-quarter-plans-for-classes-housing-health-precautions/'
  8500.0 7977.0 '0:02:48' 6891.0 0.008782 0.0083 '$0.00'
  'UCLA announces fall quarter plans for classes, housing, health precautions'
  'https://dailybruin.com/2020/06/15/ucla-announces-fall-quarter-plans-for-classes-housing-health-precautions/'
  'Mon, 15 Jun 2020 22:01:46 +0000' 'Camayak_455f5f9437'
  ' The majority of courses for the fall 2020 quarter will be taught virtually.    Around 15% to 20% of the courses will be conducted in person or as an in-person and online hybrid, according to an email to students from Executive Vice Chancellor and Provost Emily Carter. These courses include laboratory courses, performing arts courses and other courses that would be difficult to conduct online  ,   Carter said in the email.    UCLA will provide on-campus housing   to a limited number of students based on a number of factors including financial need, the email said.&nbsp;    UCLA will also pro

In [4]:
link = final.columns.get_loc("link")
views = final.columns.get_loc("views")
unique_views = final.columns.get_loc("unique_views")
entrances = final.columns.get_loc("entrances")
avg_time = final.columns.get_loc("avg_time")
bounce_rate = final.columns.get_loc("bounce_rate")
exit_percent = final.columns.get_loc("exit_percent")

def combine(one, two):
    one.reset_index(drop=True, inplace=True)
    two.reset_index(drop=True, inplace=True)
    
    one = one.to_numpy()
    two = two.to_numpy()
    
#     print(one)
    
#     print(two)


#     if one[0][link] != two[0][link]:
#         return
#     else:
    one_fraction = one[0][views] / (one[0][views] + two[0][views])
    two_fraction = two[0][views] / (one[0][views] + two[0][views])

    one[0][views] = one[0][views] + two[0][views]
    one[0][unique_views] = one[0][unique_views] + two[0][unique_views]
    one[0][entrances] = one[0][entrances] + two[0][entrances]
    one[0][avg_time] = one[0][avg_time] * one_fraction + two[0][avg_time] * two_fraction
    one[0][bounce_rate] = one[0][bounce_rate] * one_fraction + two[0][bounce_rate] * two_fraction
    one[0][exit_percent] = one[0][exit_percent] * one_fraction + two[0][exit_percent] * two_fraction

    x = pd.DataFrame(one, columns=final.columns)

    return x
    

# head = combine(final.loc[[7]], final.loc[[8]])

# head
# final.loc[[7]].to_numpy()[0][link]
    

In [5]:
def combinegroup(df):
    df.reset_index(drop=True, inplace=True)
    if len(df) == 1:
        return df
    
    
    if len(df) == 2:
        return combine(df.loc[[0]], df.loc[[1]])
    
    new = combine(df.loc[[0]], df.loc[[1]])
    for i in range(2, len(df)):
        new = combine(new, df.loc[[i]])
    
    return new

test = final.loc[final['link'] == 'https://dailybruin.com/2020/06/15/ucla-announces-fall-quarter-plans-for-classes-housing-health-precautions/']
test

har = combinegroup(test)


In [6]:
group = final.loc[final['link'] == "https://dailybruin.com/2020/03/01/whether-admissions-officers-like-applicants-social-media-or-not-shouldnt-matter/"]

har = combinegroup(group)
har



KeyError: "None of [Int64Index([0], dtype='int64')] are in the [index]"

In [8]:

links = final['link'].tolist()

def remove_duplicates(x):
  return list(dict.fromkeys(x))

links = remove_duplicates(links)

print(len(links))




748


In [9]:
aggregate = []


for i,  link in enumerate(links):
    try:
        group = final.loc[final['link'] == link]

        aggregate.append(combinegroup(group))
    except:
        print(i, link)



last = pd.concat(aggregate)
print(len(last))

748


In [10]:
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
}

for i, row in last.iterrows():
    if row.image_url == 'todo':
        try:
            req = Request(row.link, headers=headers)
            html = urlopen(req).read()
            soup = BeautifulSoup(html)

            images = soup.findAll('meta', {'property': 'og:image'})
            if len(images) == 1:
                image = images[0]['content']
                last.loc[i].image_url = image
                print(f'Processed row {i}, {image}')
            else:
                print(f'Can\'t find image: {row.link}')
        except:
            print(f'Error on {row.link}')

In [11]:
last.reset_index(drop=True, inplace=True)


last.to_csv("merged_duplicates_removed.csv")

In [12]:
test = last.head(10)

In [13]:
import numpy as np
data = pd.read_csv("merged_duplicates_removed.csv")

data.drop(data.columns[0], axis=1, inplace=True)

x = data.head(3)

y = x.to_numpy()

n = np.concatenate((y[0], y[1], ['HELLO']), axis=0)

n

array([0, '/', 90941.0, 74000.0, '0:01:24', 63825.0, 0.004599, 0.004502,
       '$0.00', 'Graduation Issue 2020',
       'https://gradissue2020.dailybruin.com/',
       'Fri, 12 Jun 2020 17:09:01 +0000', 'Camayak_180',
       "We are past the tipping point. Where do we go from here? Welcome to the Daily Bruin's 2020 Graduation Issue.",
       "['camayak_180']", "['interactive-page']",
       'gradissue2020.dailybruin.com/smol.gradissue-preview.png', 1,
       '/2020/06/04/students-sign-petition-to-fire-ucla-lecturer-after-many-find-his-email-insensitive/',
       41497.0, 38105.0, '0:04:55', 37002.0, 0.00907, 0.008916, '$0.00',
       'Students sign petition to fire UCLA lecturer after many find his email insensitive',
       'https://dailybruin.com/2020/06/04/students-sign-petition-to-fire-ucla-lecturer-after-many-find-his-email-insensitive/',
       'Thu, 04 Jun 2020 23:47:55 +0000', 'Camayak_8ce8660ac1',
       ' This post was updated June 5 at 1:35 p.m.    Editor\'s note: This post

In [14]:
def joinrows(rowzero, rowone, views_col_num):
    higher_page_views = 1;
    
    if rowzero[views_col_num] >  rowone[views_col_num]:
        higher_page_views = 0
    
    joined = np.concatenate((rowzero, rowone, [higher_page_views]), axis=0)
    
    return joined

def joindataset(data):
    
    views = data.columns.get_loc("views")
    
    cols = data.columns
    cols_zero = cols + "_zero"
    cols_one = cols + "_one"

    newcols = list(cols_zero) + list(cols_one) + ['label']

    data = data.to_numpy()

    joined_data = []
    i = len(data) - 1
    while i >= 0:
        j = i - 1
        while j >= 0:
            joined_data.append(joinrows(data[i], data[j], views))
            joined_data.append(joinrows(data[j], data[i], views))

            j -= 1
        i -= 1
        
            
    joined_df = pd.DataFrame(joined_data, columns=newcols)
    
    return joined_df

joined_data= joindataset(data)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

print(len(joined_data))

558756


In [15]:
print(len(data))

748


In [16]:
joined_data.to_csv("comparison_dataset.csv")

In [17]:
 joined_data.head()

,Unnamed: 0.1_zero,page_zero,views_zero,unique_views_zero,avg_time_zero,entrances_zero,bounce_rate_zero,exit_percent_zero,page_value_zero,title_zero,link_zero,pubDate_zero,creator_zero,encoded_text_zero,authors_zero,categories_zero,image_url_zero,Unnamed: 0.1_one,page_one,views_one,unique_views_one,avg_time_one,entrances_one,bounce_rate_one,exit_percent_one,page_value_one,title_one,link_one,pubDate_one,creator_one,encoded_text_one,authors_one,categories_one,image_url_one,label
0,747,/2020/09/22/jacks-facts-pac-12-must-embrace-ch...,22.0,19.0,0:01:55,10.0,0.007000,0.004545,$0.00,Jack's Facts: Pac-12 must embrace change to co...,https://dailybruin.com/2020/09/22/jacks-facts-...,"Tue, 22 Sep 2020 18:05:06 +0000",Camayak_ee4369071f,Its time for change in the Pac-12. Fans have ...,['cap-jack-perez'],"['sports', 'sports-columns']",NaN,746,/2020/10/21/graphic-donald-trumps-covid-19-dia...,23.0,20.0,0:02:16,11.0,0.008182,0.004783,$0.00,Graphic: Donald Trump's COVID-19 Diagnosis,https://dailybruin.com/2020/10/21/graphic-dona...,"Wed, 21 Oct 2020 18:34:56 +0000",Camayak_0570a3a4bf,,['cap-farrah-au-yeung'],['graphics'],NaN,1
1,746,/2020/10/21/graphic-donald-trumps-covid-19-dia...,23.0,20.0,0:02:16,11.0,0.008182,0.004783,$0.00,Graphic: Donald Trump's COVID-19 Diagnosis,https://dailybruin.com/2020/10/21/graphic-dona...,"Wed, 21 Oct 2020 18:34:56 +0000",Camayak_0570a3a4bf,,['cap-farrah-au-yeung'],['graphics'],NaN,747,/2020/09/22/jacks-facts-pac-12-must-embrace-ch...,22.0,19.0,0:01:55,10.0,0.007000,0.004545,$0.00,Jack's Facts: Pac-12 must embrace change to co...,https://dailybruin.com/2020/09/22/jacks-facts-...,"Tue, 22 Sep 2020 18:05:06 +0000",Camayak_ee4369071f,Its time for change in the Pac-12. Fans have ...,['cap-jack-perez'],"['sports', 'sports-columns']",NaN,0
2,747,/2020/09/22/jacks-facts-pac-12-must-embrace-ch...,22.0,19.0,0:01:55,10.0,0.007000,0.004545,$0.00,Jack's Facts: Pac-12 must embrace change to co...,https://dailybruin.com/2020/09/22/jacks-facts-...,"Tue, 22 Sep 2020 18:05:06 +0000",Camayak_ee4369071f,Its time for change in the Pac-12. Fans have ...,['cap-jack-perez'],"['sports', 'sports-columns']",NaN,745,/2020/06/11/editorial-cartoon-green-screen-gra...,24.0,24.0,0:00:33,3.0,0.003333,0.001250,$0.00,Editorial Cartoon: Green screen graduation,https://dailybruin.com/2020/06/11/editorial-ca...,"Thu, 11 Jun 2020 21:28:02 +0000",azhang3,&nbsp;,['cap-azhang3'],"['cartoons', 'opinion']",NaN,1
3,745,/2020/06/11/editorial-cartoon-green-screen-gra...,24.0,24.0,0:00:33,3.0,0.003333,0.001250,$0.00,Editorial Cartoon: Green screen graduation,https://dailybruin.com/2020/06/11/editorial-ca...,"Thu, 11 Jun 2020 21:28:02 +0000",azhang3,&nbsp;,['cap-azhang3'],"['cartoons', 'opinion']",NaN,747,/2020/09/22/jacks-facts-pac-12-must-embrace-ch...,22.0,19.0,0:01:55,10.0,0.007000,0.004545,$0.00,Jack's Facts: Pac-12 must embrace change to co...,https://dailybruin.com/2020/09/22/jacks-facts-...,"Tue, 22 Sep 2020 18:05:06 +0000",Camayak_ee4369071f,Its time for change in the Pac-12. Fans have ...,['cap-jack-perez'],"['sports', 'sports-columns']",NaN,0
4,747,/2020/09/22/jacks-facts-pac-12-must-embrace-ch...,22.0,19.0,0:01:55,10.0,0.007000,0.004545,$0.00,Jack's Facts: Pac-12 must embrace change to co...,https://dailybruin.com/2020/09/22/jacks-facts-...,"Tue, 22 Sep 2020 18:05:06 +0000",Camayak_ee4369071f,Its time for change in the Pac-12. Fans have ...,['cap-jack-perez'],"['sports', 'sports-columns']",NaN,744,/2020/10/24/students-and-faculty-make-calls-to...,26.0,25.0,0:01:30,3.0,0.006667,0.003462,$0.00,Students and Faculty Make Calls to Divest UCPD...,https://dailybruin.com/2020/10/24/students-and...,"Sat, 24 Oct 2020 20:17:07 +0000",Camayak_43d459acbe,"[embed url=""https://youtu.be/1mBpkD1YSJw"" con...",['cap-sophia-king'],['video'],NaN,1
